In [1]:

import numpy as np
from collections import Counter
import pandas as pd
import os
import time
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

class RoulettePredictor:
    def __init__(self, data):
        # Определение цветов чисел
        self.black_numbers = {2, 4, 6, 8, 10, 11, 13, 15, 17, 20, 22, 24, 26, 28, 29, 31, 33, 35}
        self.red_numbers = {1, 3, 5, 7, 9, 12, 14, 16, 18, 19, 21, 23, 25, 27, 30, 32, 34, 36}
        self.green_numbers = {0, '00'}
        
        # Преобразование данных, сохраняя различие между 0 и 00
        self.data = []
        for x in data.replace(";", ",").split(","):
            if x:
                if x == '00':
                    self.data.append('00')
                else:
                    self.data.append(int(x))
                    
        self.last_numbers = {}
        self.sequence_stats = {}
        self.total_numbers = len(self.data)
        self._build_sequences()
        self._calculate_global_stats()

        self.model = None
        self.scaler = StandardScaler()
        self.ml_features = None
        self.ml_targets = None
        self._prepare_data_for_ml()
        self._train_ml_model()

    def _get_color(self, number):
        """Определение цвета числа"""
        if number in self.black_numbers:
            return 'Черное'
        elif number in self.red_numbers:
            return 'Красное'
        else:
            return 'Зеленое'

    def _build_sequences(self):
        self.last_numbers = {}
        self.sequence_stats = {}
        
        for i in range(len(self.data) - 1):
            current_num = self.data[i]
            next_num = self.data[i + 1]
            
            if current_num not in self.last_numbers:
                self.last_numbers[current_num] = []
                self.sequence_stats[current_num] = {'total': 0, 'next_numbers': {}}
            
            self.last_numbers[current_num].append(next_num)
            self.sequence_stats[current_num]['total'] += 1
            
            if next_num not in self.sequence_stats[current_num]['next_numbers']:
                self.sequence_stats[current_num]['next_numbers'][next_num] = 0
            self.sequence_stats[current_num]['next_numbers'][next_num] += 1

    def _calculate_global_stats(self):
        self.global_stats = Counter(self.data)
        self.number_probabilities = {
            num: count / self.total_numbers * 100 
            for num, count in self.global_stats.items()
        }

    def get_number_probability(self, number):
        return self.number_probabilities.get(number, 0)

    def predict_next_numbers(self, last_number, top_n=3):
        if last_number not in self.sequence_stats:
            return self.predict_frequent_numbers(top_n)

        stats = self.sequence_stats[last_number]
        total_sequences = stats['total']
        next_numbers_stats = stats['next_numbers']

        probabilities = {
            num: (count / total_sequences * 100)
            for num, count in next_numbers_stats.items()
        }

        sorted_predictions = sorted(
            probabilities.items(),
            key=lambda x: x[1],
            reverse=True
        )[:top_n]

        return sorted_predictions
    
    def predict_with_ml(self, last_numbers, top_n=3):
        if self.model is None or not last_numbers:
            return self.predict_frequent_numbers(top_n)
        
        
        features = self._create_features(last_numbers)
        features_scaled = self.scaler.transform(features)
        
        
        probas = self.model.predict_proba(features_scaled)[0]
        
        predictions_with_proba = list(zip(self.model.classes_, probas))
        sorted_predictions = sorted(predictions_with_proba, key=lambda x: x[1], reverse=True)[:top_n]
        
        return sorted_predictions
        

    def predict_frequent_numbers(self, top_n=3):
        return [(num, self.get_number_probability(num)) 
                for num, _ in self.global_stats.most_common(top_n)]
    
    def _create_features(self, last_numbers):
        # Feature: last 3 numbers
        feature = [0] * 3
        if len(last_numbers) >= 1:
            feature[0] = last_numbers[-1] if isinstance(last_numbers[-1], int) else 0
        if len(last_numbers) >= 2:
             feature[1] = last_numbers[-2] if isinstance(last_numbers[-2], int) else 0
        if len(last_numbers) >= 3:
            feature[2] = last_numbers[-3] if isinstance(last_numbers[-3], int) else 0
        
        
        # Convert to numpy array and then to DataFrame for consistency
        return np.array(feature).reshape(1, -1)
    
    def _prepare_data_for_ml(self):
        if len(self.data) < 4:
            self.ml_features = None
            self.ml_targets = None
            return

        features = []
        targets = []
        for i in range(len(self.data) - 3):
            
            # Feature: last 3 numbers
            feature_set = [self.data[i], self.data[i+1], self.data[i+2]]
            features.append(feature_set)

            
            targets.append(self.data[i+3])

        
        self.ml_features = np.array(features)
        self.ml_targets = np.array(targets)
        
    def _train_ml_model(self):
        if self.ml_features is None or self.ml_targets is None:
            return

        if self.ml_features.size == 0 or self.ml_targets.size == 0:
            return

        
        X_reshaped = self.ml_features.reshape(self.ml_features.shape[0], -1)
        X_train, X_test, y_train, y_test = train_test_split(X_reshaped, self.ml_targets, test_size=0.2, random_state=42)
        
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        best_accuracy = 0
        best_model = None
        
        
        models = {
            'Random Forest': RandomForestClassifier(random_state=42),
            'Neural Network': MLPClassifier(random_state=42, max_iter=500),
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=200),
            'SVM': SVC(probability=True, random_state=42),
            'K-Nearest Neighbors': KNeighborsClassifier(),
            'Gaussian Naive Bayes': GaussianNB(),
            'Decision Tree': DecisionTreeClassifier(random_state=42),
            'Gradient Boosting': GradientBoostingClassifier(random_state=42)
        }
        
        for name, model in models.items():
            try:
              
              model.fit(X_train_scaled, y_train)
              y_pred = model.predict(X_test_scaled)
              accuracy = accuracy_score(y_test, y_pred)

              if accuracy > best_accuracy:
                 best_accuracy = accuracy
                 best_model = model
            except Exception as e:
                print(f"Error with {name}: {e}")


        if best_model is not None:
            self.model = best_model
            print(f"Best ML model selected with accuracy: {best_accuracy*100:.2f}%")
        else:
            print("No suitable ML model could be trained.")
    
    def analyze_patterns(self):
        # Анализ по цветам
        color_counts = {'Черное': 0, 'Красное': 0, 'Зеленое': 0}
        for num in self.data:
            if num == '00' or num == 0:
                color_counts['Зеленое'] += 1
            elif num in self.black_numbers:
                color_counts['Черное'] += 1
            elif num in self.red_numbers:
                color_counts['Красное'] += 1

        color_stats = {
            color: (count / self.total_numbers * 100)
            for color, count in color_counts.items()
        }

        # Остальной анализ
        numeric_data = [x for x in self.data if x != '00']
        df = pd.DataFrame(numeric_data, columns=['number'])
        
        even_odd = df['number'].apply(lambda x: 'Чётное' if x % 2 == 0 else 'Нечётное')
        even_odd_stats = even_odd.value_counts(normalize=True) * 100
        
        def get_range(n):
            if n == 0: return '0'
            if 1 <= n <= 12: return '1-12'
            if 13 <= n <= 24: return '13-24'
            return '25-36'
        
        range_stats = df['number'].apply(get_range).value_counts(normalize=True) * 100
        
        zero_double_count = self.data.count('00')
        zero_double_percent = (zero_double_count / self.total_numbers) * 100
        
        return {
            'colors': color_stats,
            'even_odd': even_odd_stats.to_dict(),
            'ranges': range_stats.to_dict(),
            'zero_double': zero_double_percent
        }

    def add_new_data(self, new_number):
        self.data.append(new_number)
        self.total_numbers += 1
        self._build_sequences()
        self._calculate_global_stats()
        self._prepare_data_for_ml()
        self._train_ml_model()

    def save_predictions_to_file(self, predictions, patterns, number, ml_predictions):
        with open("MajesticPredictions.txt", "a") as f:
            f.write(f"\nПрогноз после числа {number}:\n")
            f.write("-" * 40 + "\n")
            
            f.write("На основе последовательности:\n")
            for predicted_number, probability in predictions:
                f.write(f"Число {predicted_number}: {probability:.2f}% вероятность\n")
            
            f.write("\nНа основе ML модели:\n")
            for predicted_number, probability in ml_predictions:
                f.write(f"Число {predicted_number}: {probability*100:.2f}% вероятность\n")
            
            
            f.write("\nТекущая статистика:\n")
            f.write("-" * 40 + "\n")
            
            # Цвета
            f.write("Цвета:\n")
            for color, percent in patterns['colors'].items():
                f.write(f"{color}: {percent:.2f}%\n")
            
            # Чётные/Нечётные
            f.write("\nЧётные/Нечётные:\n")
            for category, percent in patterns['even_odd'].items():
                f.write(f"{category}: {percent:.2f}%\n")
            
            # Диапазоны
            f.write("\nДиапазоны:\n")
            for range_name, percent in sorted(patterns['ranges'].items()):
                if range_name == '0':
                    f.write(f"0: {percent:.2f}%\n")
                else:
                    f.write(f"{range_name}: {percent:.2f}%\n")
            
            # Вероятность 00
            f.write(f"\nВероятность '00': {patterns['zero_double']:.2f}%\n")
            f.write("-" * 40 + "\n")


def clear_screen():
    """Очистка экрана"""
    os.system('cls' if os.name == 'nt' else 'clear')

def print_statistics(number, predictor):
    """Вывод статистики для числа"""
    # Очистка экрана
    clear_screen()
    
    # Получаем предсказания
    predictions = predictor.predict_next_numbers(number)
    ml_predictions = predictor.predict_with_ml(predictor.data[-3:])
    
    # Вывод прогноза
    print(f"\nПрогноз после числа {number}:")
    print("-" * 40)
    
    print("На основе последовательности:")
    for predicted_number, probability in predictions:
        print(f"Число {predicted_number}: {probability:.2f}% вероятность")
    
    print("\nНа основе ML модели:")
    for predicted_number, probability in ml_predictions:
        print(f"Число {predicted_number}: {probability*100:.2f}% вероятность")

    # Получаем текущую статистику
    patterns = predictor.analyze_patterns()
    
    # Запись в файл
    predictor.save_predictions_to_file(predictions, patterns, number, ml_predictions)

    # Вывод статистики
    print("\nТекущая статистика:")
    print("-" * 40)
    
    # Цвета
    print("Цвета:")
    for color, percent in patterns['colors'].items():
        print(f"{color}: {percent:.2f}%")
    
    # Чётные/Нечётные
    print("\nЧётные/Нечётные:")
    for category, percent in patterns['even_odd'].items():
        print(f"{category}: {percent:.2f}%")
    
    # Диапазоны
    print("\nДиапазоны:")
    for range_name, percent in sorted(patterns['ranges'].items()):
        if range_name == '0':
            print(f"0: {percent:.2f}%")
        else:
            print(f"{range_name}: {percent:.2f}%")
    
    # Вероятность 00
    print(f"\nВероятность '00': {patterns['zero_double']:.2f}%")


# Основной код
data = "8;24;22;32;2;23;11;29;9;8;16;1;18;34;1;14;5;34;25;7;8;26;2;10;5;4;32;29;5;10;20;8;35;13;15;29;11;4;29;1;14;33;31;13;11;3;8;23;4;11;26;29;18;00;31;7;31;1;20;22;8;34;10;6;4;15;4;24;4;6;34;11;24;9;22;27;34;00;20;9;25;20;11;27;20;6;24;9;5;35;35;14;35;5;3;0;13;30;24;14;25;17;25;26;16;2;26;34;11;17;11;11;4;22;15;28;33;17;22;20;10;16;0;3;14;27;4;12;24;10;19;2;6;11;4;29;30;10;35;12;18;34;32;6;30;12;24;12;1;14;12;2;5;28;24;19;31;36;19;32;14;21;22;2;14;29;8;6;31;29;6;15;11;8;2;21;14;30;6;11;31;31;16;12;25;00;18;34;16;6;32;34;1;0;17;25;23;33;35;6;36;4;27;23;30;34;15;14;13;30;10;10;14;9;5;31;36;27;18;7;12;3;17;26;11;4;18;32;13"
predictor = RoulettePredictor(data)

while True:
    try:
        last_number = input("\nВведите последнее выпавшее число (или 'q' для выхода): ")
        if last_number.lower() == 'q':
            break

        # Проверка и преобразование ввода
        if last_number == '00':
            number = '00'
        else:
            number = int(last_number)
            if not (0 <= number <= 36):
                raise ValueError("Число должно быть от 0 до 36 или '00'")

        # Добавляем новое число в датасет
        predictor.add_new_data(number)
        
        # Выводим статистику
        print_statistics(number, predictor)

    except ValueError as e:
        print(f"Ошибка: {str(e)}")
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")


c:\Users\1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best ML model selected with accuracy: 8.70%


c:\Users\1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best ML model selected with accuracy: 13.04%

Прогноз после числа 5:
----------------------------------------
На основе последовательности:
Число 34: 14.29% вероятность
Число 4: 14.29% вероятность
Число 10: 14.29% вероятность

На основе ML модели:
Число 1: 18.21% вероятность
Число 11: 11.09% вероятность
Число 29: 11.00% вероятность

Текущая статистика:
----------------------------------------
Цвета:
Черное: 53.91%
Красное: 43.48%
Зеленое: 2.61%

Чётные/Нечётные:
Чётное: 56.39%
Нечётное: 43.61%

Диапазоны:
0: 1.32%
1-12: 39.65%
13-24: 29.07%
25-36: 29.96%

Вероятность '00': 1.30%


c:\Users\1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


...

Best ML model selected with accuracy: 8.70%

Прогноз после числа 19:
----------------------------------------
На основе последовательности:
Число 2: 33.33% вероятность
Число 31: 33.33% вероятность
Число 32: 33.33% вероятность

На основе ML модели:
Число 4: 9.74% вероятность
Число 5: 8.34% вероятность
Число 34: 6.55% вероятность

Текущая статистика:
----------------------------------------
Цвета:
Черное: 53.22%
Красное: 44.21%
Зеленое: 2.58%

Чётные/Нечётные:
Чётное: 55.65%
Нечётное: 44.35%

Диапазоны:
0: 1.30%
1-12: 40.00%
13-24: 29.13%
25-36: 29.57%

Вероятность '00': 1.29%


c:\Users\1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best ML model selected with accuracy: 10.64%

Прогноз после числа 32:
----------------------------------------
На основе последовательности:
Число 2: 16.67% вероятность
Число 29: 16.67% вероятность
Число 6: 16.67% вероятность

На основе ML модели:
Число 0: 64.74% вероятность
Число 31: 3.81% вероятность
Число 5: 3.62% вероятность

Текущая статистика:
----------------------------------------
Цвета:
Черное: 52.99%
Красное: 44.44%
Зеленое: 2.56%

Чётные/Нечётные:
Чётное: 55.84%
Нечётное: 44.16%

Диапазоны:
0: 1.30%
1-12: 39.83%
13-24: 29.00%
25-36: 29.87%

Вероятность '00': 1.28%


c:\Users\1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best ML model selected with accuracy: 10.64%

Прогноз после числа 36:
----------------------------------------
На основе последовательности:
Число 19: 33.33% вероятность
Число 4: 33.33% вероятность
Число 27: 33.33% вероятность

На основе ML модели:
Число 12: 10.01% вероятность
Число 14: 9.93% вероятность
Число 11: 8.79% вероятность

Текущая статистика:
----------------------------------------
Цвета:
Черное: 52.77%
Красное: 44.68%
Зеленое: 2.55%

Чётные/Нечётные:
Чётное: 56.03%
Нечётное: 43.97%

Диапазоны:
0: 1.29%
1-12: 39.66%
13-24: 28.88%
25-36: 30.17%

Вероятность '00': 1.28%
Ошибка: invalid literal for int() with base 10: ''
